In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [13]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv("Sources/test_final_100k.csv")

In [14]:
test = pd.concat([test,postulaciones])
test = test.drop_duplicates(['idaviso','idpostulante'])
postulaciones = test[100000:].drop('id', axis=1)
test = test[:100000].drop('id', axis=1)

In [15]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)
postulaciones.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,1112248724,NjlD,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,33,206236,37.0,FEM,12
1,1112248724,MV6eM8N,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,33,189555,27.0,MASC,11
2,1112304156,MV6eM8N,1.0,Full-time,Junior,Comunicacion,RH Talentum,34,189555,27.0,MASC,11
3,1112445502,MV6eM8N,1.0,Full-time,Senior / Semi-Senior,Comunicacion,Jones Lang LaSalle,34,189555,27.0,MASC,11
4,1112254843,MV6eM8N,1.0,Full-time,Senior / Semi-Senior,E-commerce,ASSIST CARD,64,189555,27.0,MASC,11


In [16]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test['sepostulo']=1
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,739260,6M9ZQR,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,69266,42.0,FEM,21
1,739260,6v1xdL,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,83910,31.0,MASC,9
2,739260,ezRKm9,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,351736,36.0,FEM,12
3,758580,1Q35ej,1,Full-time,Otro,Tecnologia / Sistemas,TeleTech,16,15447,69.0,MASC,15
4,758580,EAN4J6,1,Full-time,Otro,Tecnologia / Sistemas,TeleTech,16,138590,32.0,FEM,11


In [17]:
resultado = postulaciones['sepostulo']
resultadob = test['sepostulo']
postulaciones.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,69266,42.0,FEM,21
1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,83910,31.0,MASC,9
2,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,351736,36.0,FEM,12
3,Full-time,Otro,Tecnologia / Sistemas,TeleTech,16,15447,69.0,MASC,15
4,Full-time,Otro,Tecnologia / Sistemas,TeleTech,16,138590,32.0,FEM,11


In [18]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in postulaciones.columns:
    if not postulaciones[col].dtype == 'O': continue
    print(col)
    postulaciones[col] = postulaciones[col].fillna('na')
    test[col] = test[col].fillna('na')
    le.fit(pd.concat([postulaciones[col],test[col]]))
    test[col] = le.transform(test[col])
    postulaciones[col] = le.transform(postulaciones[col])
postulaciones.head()

tipo_de_trabajo
nivel_laboral
nombre_area
denominacion_empresa
sexo


,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,1,4,120,2428,33,206236,37.0,1,12
1,1,4,120,2428,33,189555,27.0,2,11
2,1,2,36,3236,34,189555,27.0,2,11
3,1,4,36,2228,34,189555,27.0,2,11
4,1,4,64,154,64,189555,27.0,2,11


In [19]:
postulaciones.sample(5)
from sklearn.utils import shuffle
postulaciones = shuffle(postulaciones)

In [20]:
postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [21]:
aviso = None
postulantes = None
postulaciones.fillna(-1,inplace=True)
test.fillna(-1,inplace=True)
test.drop([6],axis=1,inplace=True)
postulaciones.drop([6],axis=1,inplace=True)

In [ ]:
clf = BernoulliNB()
clf.fit(postulaciones,resultado)
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
y_pred = clf.predict(test)

In [ ]:
test = pd.read_csv('Sources/test_final_100k.csv')
test['sepostulo'] = y_pred
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)
test.to_csv('Sources/resultados_random.csv', index=False)

y_pred[y_pred==1].shape

